In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import re
import os
import glob
import math

In [ ]:
file_path = "fidelity_gen_custom_portfolio.csv"
df = pd.read_csv(
    os.path.join(os.path.expanduser('~'),
                 'work', 'Downloads', 'finances', file_path),
                delimiter='|')
df.head()

In [ ]:
def parse_currency(value):
    return float(re.sub(r'[^0-9.]', '', value))

df["Balance"] = df["Balance"].apply(parse_currency)

total_balance = df["Balance"].sum()
title_text = f"Fund Distribution (Total: ${total_balance:,.2f})"
df_sorted = df.sort_values("Balance", ascending=True).reset_index(drop=True)

In [ ]:
fig = px.treemap(
    df,
    path=["Fund"],
    values="Balance",
    title=title_text,
    color="Balance",
    color_continuous_scale="Blues",
    hover_data={"Balance": True},
)

fig.update_traces(textinfo="label+value")
fig.show()

In [ ]:
fig_bar = px.bar(
    df_sorted,
    x="Balance",
    y="Fund",
    orientation="h",
    title=title_text,
    text_auto=True,
    color="Balance",
    color_continuous_scale="Blues",
)
fig_bar.update_layout(yaxis_title="", xaxis_title="Balance ($)", yaxis_showticklabels=False)
fig_bar.show()

In [ ]:
df['Fund']

In [ ]:
dfm = pd.read_csv(
    os.path.join(os.path.expanduser('~'),
                 'work', 'Downloads', 'finances', 'Portfolio_Positions_Mar-30-2025.csv'),
                delimiter=',')
dfm.head()

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
dfm_filtered = dfm["Symbol"].dropna()
dfm_filtered[dfm_filtered.str.match(r'^[A-Z]{1,5}(\.[A-Z])?$')]

In [ ]:
remove_list = ['ARKG', 'ARKQ', 'BLDR', 'CRSP', 'FNILX', 'FROG', 'FSLY', 'FXAIX', 'FZILX', 'FZROX', 'NET', 'QQQ', 'SENS', 'SNOW', 'SOFI', 'BITB', 'TWLO']
dfm[dfm["Symbol"].isin(remove_list)][["Symbol", "Total Gain/Loss Dollar"]]

## view asset allocation by account type FIDELITY

In [ ]:
ticker_to_category = {
    "VOO": "CORE",
    "DIA": "CORE",
    "IVV": "CORE",
    "SPY": "CORE",
    "VXUS": "INTL",
    "VWO": "INTL",
    "QQQ": "GROWTH",
    "QQQM": "GROWTH",
    "VGT": "GROWTH",
    "VUG": "GROWTH",
    "VNQ": "GROWTH",
    "VOOG": "GROWTH",
    "VYM": "GROWTH",
    "BITB": "GROWTH",
    "XSHD": "GROWTH",
    "SCHD": "STABLE",
    "AMD": "INDIVL",
    "AAPL": "INDIVL",
    "BLDR": "INDIVL",
    "GOOGL": "INDIVL",
    "HD": "INDIVL",
    "JPM": "INDIVL",
    "META": "INDIVL",
    "NVDA": "INDIVL",
    "JNJ": "INDIVL",
    "PG": "INDIVL",
    "SOFI": "INDIVL",
    "T": "INDIVL",
    "TGT": "INDIVL",
}

export_date = 'Apr-29-2025'

files = glob.glob(
    os.path.join(os.path.expanduser('~'),
                 'work', 'Downloads', 'finances', f'Portfolio_Positions_{export_date}*.csv'),
    )

dfs = [pd.read_csv(file) for file in files]
combined = pd.concat(dfs)

combined = combined.dropna(subset=["Symbol", "Current Value"])  # Remove cash and empty rows
combined["Current Value"] = combined["Current Value"].replace('[\$,]', '', regex=True).astype(float)
combined["Symbol"] = combined["Symbol"].str.strip()

# Map to categories
combined["Category"] = combined["Symbol"].map(ticker_to_category)
combined = combined.dropna(subset=["Category"])  # drop unmatched

# ---------------------
# OVERALL PIE CHART
# ---------------------
overall_summary = (
    combined.groupby("Category")
    .agg({
        "Current Value": "sum",
        "Symbol": lambda x: ", ".join(sorted(set(x)))
    })
    .reset_index()
)

overall_summary = overall_summary.rename(columns={"Symbol": "Tickers"})
total_value = overall_summary["Current Value"].sum()
formatted_total = f"${total_value:,.2f}"

fig_overall = px.pie(
    overall_summary,
    names="Category",
    values="Current Value",
    title=f"TOTAL Portfolio Breakdown (All Accounts) — {formatted_total}",
    hole=0.4,
    hover_data=["Tickers"]
)

fig_overall.update_traces(textinfo='percent+label')


# Prep per-account grouped data
facet_summary = (
    combined.groupby(["Account Name", "Category"])
    .agg({
        "Current Value": "sum",
        "Symbol": lambda x: ", ".join(sorted(set(x)))
    })
    .reset_index()
    .rename(columns={"Symbol": "Tickers"})
)

account_totals = (
    combined.groupby("Account Name")["Current Value"]
    .sum()
    .apply(lambda x: f"${x:,.2f}")
    .to_dict()
)

# Format titles: "Account Name — $Total"
accounts = facet_summary["Account Name"].unique()
subplot_titles = [f"{acct} — {account_totals[acct]}" for acct in accounts]
n_accounts = len(accounts)
n_cols = 2
n_rows = math.ceil(n_accounts / n_cols)

# Create subplots grid
fig_faceted = make_subplots(
    rows=n_rows,
    cols=n_cols,
    specs=[[{"type": "domain"}] * n_cols for _ in range(n_rows)],
    subplot_titles=subplot_titles,
    vertical_spacing=0.08  # ~1.25cm depending on screen; tweak as needed
)

# Add pies
for i, account in enumerate(accounts):
    row = i // n_cols + 1
    col = i % n_cols + 1
    df = facet_summary[facet_summary["Account Name"] == account]

    fig_faceted.add_trace(
        go.Pie(
            labels=df["Category"],
            values=df["Current Value"],
            textinfo="percent+label",
            hovertemplate="<b>%{label}</b><br>Value: %{value:$,.2f}<br>Tickers: %{customdata}",
            customdata=df["Tickers"]
        ),
        row=row,
        col=col
    )

# Format
fig_faceted.update_layout(
    height=350 * n_rows,
    title_text="Portfolio Breakdown by Account (Facetted 2xN Layout)",
    title_x=0.5,
    showlegend=False,
    margin=dict(t=60, b=40, l=20, r=20)
)

fig_overall.show()
fig_faceted.show()

In [ ]:
selected_cols = ['Account Name', 'Symbol', 'Quantity']
df_fid = pd.concat([df[selected_cols] for df in dfs], ignore_index=True)
df_fid = df_fid[~((df_fid['Symbol'] == 'SPAXX**') | (df_fid['Quantity'].isna()))]

In [ ]:
print(df_fid)

In [ ]:
formatted_date = datetime.strptime(export_date, '%b-%d-%Y')
formatted_date = formatted_date.strftime('%Y-%m-%d')
file_path = os.path.join(os.path.expanduser('~'),
                 'work', 'Downloads', 'finances', f'Roth_Individual_401(k)-Positions-{formatted_date}-*.csv')

matching_files = glob.glob(file_path)

if not matching_files:
    raise FileNotFoundError(f"No files found matching pattern: {file_path}")

first_matching_file = matching_files[0]

with open(first_matching_file, 'r') as f:
    first_line = f.readline().strip()

match = re.search(r'account (.*?) \.\.\.', first_line)
account_type = match.group(1) if match else "Unknown Account"
dft = pd.read_csv(first_matching_file, skiprows=2)
df_cs = pd.DataFrame({
    'Account Name': account_type,
    'Symbol': dft['Symbol'],
    'Quantity': dft['Qty (Quantity)']
})
df_cs = df_cs[df_cs['Quantity'].str.isdigit()]
print(df_cs)

In [ ]:
combined_df = pd.concat([df_fid, df_cs], ignore_index=True)
combined_df

In [ ]:
file_path = os.path.join(os.path.expanduser('~'),
                 'work', 'Downloads', 'finances', 'enriched_portfolio_analysis.csv')
dff = pd.read_csv(file_path)
dff

## buy list summary

In [ ]:
file_path = os.path.join(os.path.expanduser('~'),
                 'work', 'Downloads', 'finances', 'portfolio_buy_list_summary.csv')
dff = pd.read_csv(file_path)
dff